In [ ]:
!pip install transformers datasets

In [ ]:
from psutil import virtual_memory
from google.colab import files
import pandas as pd
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from transformers import DataCollatorWithPadding
from transformers import TFAutoModel
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from datasets import Dataset
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
uploaded = files.upload()

Saving train_task1_robertuito.csv to train_task1_robertuito.csv
Saving val_task1_robertuito.csv to val_task1_robertuito.csv


In [ ]:
train_df = pd.read_csv(io.BytesIO(uploaded['train_task1_robertuito.csv']))
val_df = pd.read_csv(io.BytesIO(uploaded['val_task1_robertuito.csv']))

In [ ]:
train_df

,Unnamed: 0,train_texts,train_labels
0,0,¿Quién es Mariquita Pérez? @usuario le cuenta ...,1
1,1,@usuario Si emoji luna nueva con cara emoji m...,2
2,2,"Cuando le preguntó a mi papá, qué haría si fue...",0
3,3,"""como si es joto"" SLAY QUEEN url",0
4,4,LOL. ¿Les había dicho que todas las reglas par...,0
...,...,...,...
5595,5595,Y así fue la gira de trabajo el día de hoy a l...,1
5596,5596,"@usuario Nono, yo SI quiero ser abiertamente t...",0
5597,5597,"Que puta hueva tengobde entrenar, pero aqui es...",1
5598,5598,Se me ocurrió preguntarle a mi ex marido si él...,1


In [ ]:
train_texts, val_texts, train_labels, val_labels = list(train_df['train_texts']), list(val_df['val_texts']), list(train_df['train_labels']), list(val_df['val_labels'])

In [ ]:
def get_class_weight(cl):
  n_samples = len(train_labels)
  n_classes = 3
  n_samples_class = train_labels.count(cl)

  weight = n_samples / (n_classes * n_samples_class)
  return weight

In [ ]:
class_weight = {0: get_class_weight(0), 1: get_class_weight(1), 2: get_class_weight(2)}

In [ ]:
class_weight

{0: 0.5348615090735435, 1: 1.32858837485172, 2: 2.647754137115839}

In [ ]:
# Utilizamos un conjunto de pesos que no sesgue en exceso el proceso de entrenamiento.
class_weight = {0: 0.7, 1: 1, 2: 1.3}

In [ ]:
class_weight

{0: 0.7, 1: 1, 2: 1.3}

In [ ]:
print(len(train_texts))
print(len(val_texts))

5600
1400


In [ ]:
checkpoint = "pysentimiento/robertuito-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
train_encodings['labels'] = train_labels
val_encodings['labels'] = val_labels

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
batch_size = 4
num_epochs = 4

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = Dataset.from_dict(train_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size
)

tf_validation_dataset = Dataset.from_dict(val_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=2e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = AdamW(learning_rate=lr_scheduler, amsgrad=False)
loss=SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3,
                                                             attention_probs_dropout_prob=0.1,
                                                             hidden_dropout_prob=0.2,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=["accuracy"]
)

In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    class_weight=class_weight
)

Epoch 1/4
1400/1400 [==============================] - 412s 259ms/step - loss: 0.4016 - accuracy: 0.8393 - val_loss: 0.3258 - val_accuracy: 0.8779
Epoch 2/4
1400/1400 [==============================] - 335s 240ms/step - loss: 0.2139 - accuracy: 0.9170 - val_loss: 0.3775 - val_accuracy: 0.8679
Epoch 3/4
1400/1400 [==============================] - 334s 239ms/step - loss: 0.1242 - accuracy: 0.9570 - val_loss: 0.3889 - val_accuracy: 0.8843
Epoch 4/4
1400/1400 [==============================] - 335s 240ms/step - loss: 0.0707 - accuracy: 0.9762 - val_loss: 0.4282 - val_accuracy: 0.8779


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("cybersyn/robertuito-homomex-track1", from_pt=True)

In [ ]:
tokenizer.push_to_hub("cybersyn/robertuito-homomex-track1")

CommitInfo(commit_url='https://huggingface.co/cybersyn/robertuito-homomex-track1/commit/c244836603c9f0eaeaad0073450d38ae78b20448', commit_message='Upload tokenizer', commit_description='', oid='c244836603c9f0eaeaad0073450d38ae78b20448', pr_url=None, pr_revision=None, pr_num=None)

###Download and test de model

In [ ]:
del model

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("cybersyn/robertuito-homomex-track1", num_labels=3)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cybersyn/robertuito-homomex-track1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)

350/350 [==============================] - 38s 69ms/step


In [ ]:
print(classification_report(class_preds, val_encodings['labels']))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       853
           1       0.91      0.88      0.90       386
           2       0.67      0.65      0.66       161

    accuracy                           0.88      1400
   macro avg       0.83      0.82      0.82      1400
weighted avg       0.88      0.88      0.88      1400



In [ ]:
uploaded = files.upload()

Saving pred_task1_robertuito.csv to pred_task1_robertuito.csv


In [ ]:
pred_df = pd.read_csv(io.BytesIO(uploaded['pred_task1_robertuito.csv']))

In [ ]:
pred_df

,Unnamed: 0,0
0,0,sera que mi perrita es lesbiana?:O
1,1,"""@usuario: Quiero una amiga lencha"" te vaz a m..."
2,2,"Ash !! La jotita del programa ""hoy"" no deja de..."
3,3,En una boda gay ¿Les avientan arroz con popote?
4,4,sos mas facil que la tabla del 1 PUTA!
...,...,...
3995,3995,El tema es que la diversidad y todos estos tem...
3996,3996,Acerca de las mujeres trans en el deporte. \nH...
3997,3997,"""¡Siii Ophelia! Entiende que las mujeres siemp..."
3998,3998,"En los años setentas, varias artistas famosas ..."


In [ ]:
pred_texts = list(pred_df['0'])

In [ ]:
checkpoint = "pysentimiento/robertuito-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
pred_encodings = tokenizer(pred_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
pred_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
batch_size = 8

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_prediction_dataset = Dataset.from_dict(pred_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size
)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
preds = model.predict(tf_prediction_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)

1000/1000 [==============================] - 83s 80ms/step


In [ ]:
class_preds

array([0, 0, 2, ..., 0, 0, 0])

In [ ]:
def assign_label(num):
  if num == 0:
    return 'NP'
  elif num == 1:
    return 'NA'
  return 'P'

In [ ]:
final_list = [assign_label(x) for x in class_preds]

In [ ]:
pred_df['label'] = final_list

In [ ]:
pred_df.columns

Index(['Unnamed: 0', '0', 'label'], dtype='object')

In [ ]:
pred_df['TaskName'] = 'LGBTphobiaDetectionMultiClass'

In [ ]:
pred_df.rename(columns={'Unnamed: 0':'IdentifierOfATweet', 'label':'Class'}, inplace=True)
del pred_df['0']

In [ ]:
pred_df = pred_df[['TaskName','IdentifierOfATweet','Class']]

In [ ]:
pred_df

,TaskName,IdentifierOfATweet,Class
0,LGBTphobiaDetectionMultiClass,0,NP
1,LGBTphobiaDetectionMultiClass,1,NP
2,LGBTphobiaDetectionMultiClass,2,P
3,LGBTphobiaDetectionMultiClass,3,NP
4,LGBTphobiaDetectionMultiClass,4,NA
...,...,...,...
3995,LGBTphobiaDetectionMultiClass,3995,NP
3996,LGBTphobiaDetectionMultiClass,3996,NP
3997,LGBTphobiaDetectionMultiClass,3997,NP
3998,LGBTphobiaDetectionMultiClass,3998,NP


In [ ]:
import csv
pred_df.to_csv('./track1_submission.txt', header=False, sep='\t', index=False, lineterminator='\n\n', quotechar='"', quoting=csv.QUOTE_ALL)

The best result in my opinion needed to decrease a bit the f1-score in order to improve the accuracy of the minority class (lgtb phobic tweets) as we can safely assume those are the most important to be predicted. In order to achieve that, I've trained the model with the following settings/hyperparameters:

- Class weights: add some "importance" to minority class and subtract some  to largest class.
- learning rate scheduler with linear decrease starting at 5e-5.
- batch size of 8 tweets
- 4 epochs
- AMSGrad variant of Adam optimizer.